In [1]:
import os
import gc
import warnings
import logging
import time
import math
import cv2
from pathlib import Path
import joblib


import numpy as np
import pandas as pd
import librosa
import soundfile as sf
from soundfile import SoundFile 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import autocast, GradScaler
import timm
from tqdm.auto import tqdm
from glob import glob
import torchaudio
import random
import itertools
from typing import Union

import pickle
import torchaudio
import torchaudio.transforms as AT
from contextlib import contextmanager
import concurrent.futures


warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.ERROR)





In [2]:
class CFG:
 
    taxonomy_csv = '/kaggle/input/birdclef-2025/taxonomy.csv'

    model_dicts = {
    "efficientnet_b0":[
                       '/kaggle/input/effinetb0-all-2025/model_fold1.pth' ,
        '/kaggle/input/effnetb0seed2024/model_fold0.pth',
        '/kaggle/input/effientb0-2023/model_fold0.pth'],


    # "tf_mobilenetv3_large_100.in1k":["/kaggle/input/mobile-bcefocal/model_fold0.pth",
    #                                  '/kaggle/input/mobile-bcefocal/model_fold1.pth',
    #                                  "/kaggle/input/mobile-bcefocal/model_fold2.pth"
    #                                 ]
        
    # "regnety_008":
        # "/kaggle/input/bird25-all-train-effnet-v2-s-bcefocalloss/model_fold1.pth"    
}

    model_path = "/kaggle/input/effinetb0-all-2025/model_fold4.pth" 
    model_name = "efficientnet_b0"
    # # Audio parameters
    # FS = 32000  
    # WINDOW_SIZE = 5  
    
    # # Mel spectrogram parameters
    # N_FFT = 1024
    # HOP_LENGTH = 512
    # N_MELS = 128
    # FMIN = 50
    # FMAX = 16000
    # TARGET_SHAPE = (256, 256)
    num_classes = 206
  
    in_channels = 1
    device = 'cpu'  
    
    # Inference parameters
    batch_size = 32  
    # TTA 的次数。 如果 use_tta 为 True，则指定对每个测试样本进行多少次增强。
    threshold = 0.5

    debug =  True
    # True  False
    debug_count = 3

cfg = CFG()
print(f"Using device: {cfg.device}")
print(f"Loading taxonomy data...")
taxonomy_df = pd.read_csv(cfg.taxonomy_csv)
species_ids = taxonomy_df['primary_label'].tolist()

Using device: cpu
Loading taxonomy data...


In [3]:
class Efficientb0Model(nn.Module):
    def __init__(self,cfg,model_name):
        super().__init__()
        self.cfg = cfg
        
        self.backbone = timm.create_model(
            model_name,
            pretrained=False,  
            in_chans=cfg.in_channels,
            drop_rate=0.0,       # 训练时就不需要使用暂退法来类似进行正则化了
            drop_path_rate=0.0
        )
        
        if 'efficientnet' in model_name:
            backbone_out = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Identity()
        elif 'resnet' in model_name:
            backbone_out = self.backbone.fc.in_features
            self.backbone.fc = nn.Identity()
        else:
            backbone_out = self.backbone.get_classifier().in_features
            self.backbone.reset_classifier(0, '')
        
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.feat_dim = backbone_out
        self.classifier = nn.Linear(backbone_out, cfg.num_classes)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        features = self.backbone(x)
        
        if isinstance(features, dict):
            features = features['features']
            
        if len(features.shape) == 4:
            features = self.pooling(features)
            features = features.view(features.size(0), -1)
        
        logits = self.classifier(features)
        
        return logits

In [4]:
! python -m pip install --no-index --find-links=../input/openvino-wheels -r ../input/openvino-wheels/requirements.txt

Looking in links: ../input/openvino-wheels
Processing /kaggle/input/openvino-wheels/openvino_dev-2024.6.0-17404-py3-none-any.whl (from openvino-dev[onnx]==2024.6.0->-r ../input/openvino-wheels/requirements.txt (line 1))
Processing /kaggle/input/openvino-wheels/networkx-3.1-py3-none-any.whl (from openvino-dev==2024.6.0->openvino-dev[onnx]==2024.6.0->-r ../input/openvino-wheels/requirements.txt (line 1))
Processing /kaggle/input/openvino-wheels/openvino_telemetry-2025.1.0-py3-none-any.whl (from openvino-dev==2024.6.0->openvino-dev[onnx]==2024.6.0->-r ../input/openvino-wheels/requirements.txt (line 1))
Processing /kaggle/input/openvino-wheels/openvino-2024.6.0-17404-cp311-cp311-manylinux2014_x86_64.whl (from openvino-dev==2024.6.0->openvino-dev[onnx]==2024.6.0->-r ../input/openvino-wheels/requirements.txt (line 1))
Processing /kaggle/input/openvino-wheels/fastjsonschema-2.17.1-py3-none-any.whl (from openvino-dev[onnx]==2024.6.0->-r ../input/openvino-wheels/requirements.txt (line 1))
  Att

In [5]:
from openvino.tools import mo # 用于模型转换
import openvino as ov
from openvino.runtime import Core # 用于模型加载和推理




def convert_pytorch_to_openvino(pytorch_model, model_name, output_dir, example_input_shape,i):
    """
    将 PyTorch 模型转换为 OpenVINO IR 格式。

    Args:
        pytorch_model: 一个已训练的 PyTorch 模型实例。
        model_name (str): 目标 OpenVINO 模型文件的名称 (例如 'my_model')。
        output_dir (str): 保存 .xml 和 .bin 文件的目录。
        example_input_shape (tuple): 模型期望的输入形状 (例如 (batch_size, channels, height, width))。
    """
    os.makedirs(output_dir, exist_ok=True)
    onnx_path = os.path.join(output_dir, f"{model_name}.onnx")
    xml_path = os.path.join(output_dir, f"{model_name}.xml")
    
    pytorch_model.eval()
    pytorch_model.cpu() 

    print(f"将 PyTorch 模型 '{model_name}' 导出为 ONNX...")
    dummy_input = torch.randn(example_input_shape)
    
    try:
        torch.onnx.export(pytorch_model,
                          dummy_input,
                          onnx_path,
                          verbose=False,
                          opset_version=13, # 选择合适的 ONNX opset 版本
                          input_names=['input_0'], # 输入层名称
                          output_names=['output_0'], # 输出层名称
                          dynamic_axes={'input_0': {0: 'batch_size'}} # 如果需要动态 batch size
                         )
        print(f"ONNX 模型已保存到: {onnx_path}")
    except Exception as e:
        print(f"导出 ONNX 失败: {e}")
        return None

    # 2. 使用 Model Optimizer 将 ONNX 模型转换为 OpenVINO IR 格式
    print(f"使用 Model Optimizer 将 ONNX 模型转换为 OpenVINO IR...")
    try:
        ov_model = mo.convert_model(onnx_path, 
                                    compress_to_fp16=True # 可以选择压缩为 FP16 精度以减小模型大小和提高推理速度
                                   )
        
        # 保存 OpenVINO IR 模型
        ov.save_model(ov_model,xml_path)
        print(f"OpenVINO IR 模型已保存到: {xml_path} 和 {Path(xml_path).with_suffix('.bin')}")
        return xml_path
    except Exception as e:
        print(f"转换为 OpenVINO IR 失败: {e}")
        return None

In [6]:
def load_openvino_model(xml_path, device="CPU"):
    """
    加载 OpenVINO IR 模型并编译。

    Args:
        xml_path (str): OpenVINO IR 模型的 .xml 文件路径。
        device (str): 推理设备
    Returns:
        openvino.runtime.CompiledModel: 编译后的 OpenVINO 模型对象。
    """
    core = Core()
    model = core.read_model(model=xml_path)
    
    # 编译模型以优化到指定设备
    compiled_model = core.compile_model(model=model, device_name=device)
    print(f"OpenVINO 模型 '{Path(xml_path).stem}' 已编译到设备: {device}")
    return compiled_model

In [7]:
# one example

if __name__ == "__main__":
    for model_name,model_paths in cfg.model_dicts.items():
        for i,model_path in enumerate(model_paths):
            dummy_model = Efficientb0Model(cfg,model_name)
            checkpoint = torch.load(model_path, map_location='cpu',weights_only = False)
            dummy_model.load_state_dict(checkpoint['model_state_dict']) # 加载训练好的权重
        
            output_ir_path = convert_pytorch_to_openvino(
                dummy_model, 
                model_name, 
                output_dir=f"/kaggle/working/model_{i}", 
                example_input_shape=(1, 1, 256,256) ,
                i=i
            )
            if output_ir_path:
                print(f"模型转换成功，IR路径: {output_ir_path}")
            else:
                print("模型转换失败。")
    print("finish")

    


将 PyTorch 模型 'efficientnet_b0' 导出为 ONNX...
ONNX 模型已保存到: /kaggle/working/model_0/efficientnet_b0.onnx
使用 Model Optimizer 将 ONNX 模型转换为 OpenVINO IR...
[ INFO ] MO command line tool is considered as the legacy conversion API as of OpenVINO 2023.2 release.
In 2025.0 MO command line tool and openvino.tools.mo.convert_model() will be removed. Please use OpenVINO Model Converter (OVC) or openvino.convert_model(). OVC represents a lightweight alternative of MO and provides simplified model conversion API. 
Find more information about transition from MO to OVC at https://docs.openvino.ai/2023.2/openvino_docs_OV_Converter_UG_prepare_model_convert_model_MO_OVC_transition.html
OpenVINO IR 模型已保存到: /kaggle/working/model_0/efficientnet_b0.xml 和 /kaggle/working/model_0/efficientnet_b0.bin
模型转换成功，IR路径: /kaggle/working/model_0/efficientnet_b0.xml
将 PyTorch 模型 'efficientnet_b0' 导出为 ONNX...
ONNX 模型已保存到: /kaggle/working/model_1/efficientnet_b0.onnx
使用 Model Optimizer 将 ONNX 模型转换为 OpenVINO IR...
[ INFO ] MO 